# Load and Pre-Process Data

In [ ]:
import tensorflow as tf
import seaborn as sns
import numpy as np
from PIL import Image
import glob
from collections import defaultdict
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
IMG_SIZE = (94, 125)

def pixels_from_path(file_path):
    im = Image.open(file_path)

    im = im.resize(IMG_SIZE)
    np_im = np.array(im)
    # Matrix of pixel RGB values

    return np_im

shape_counts = defaultdict(int)

for i, cat in enumerate(glob.glob('cats/*')[:1000]):
    if i%100 == 0:
        print(i)
    img_shape = pixels_from_path(cat).shape
    shape_counts[str(img_shape)]= shape_counts[str(img_shape)]+ 1

shape_items = list(shape_counts.items())
shape_items.sort(key = lambda x: x[1])
shape_items.reverse()

# 10% of the data will automatically be used for validation
validation_size = 0.1
img_size = IMG_SIZE # resize images to be 374x500 (most common shape)
num_channels = 3 # RGB
sample_size = 8192 # We'll use 8192 pictures (2**13)
pixels_from_path(glob.glob('cats/*')[5]).shape

SAMPLE_SIZE = 2048

print("loading training cat images...")
cat_train_set = np.asarray([pixels_from_path(cat) for cat in glob.glob('cats/*')[:SAMPLE_SIZE]])

print("loading training dog images...")
dog_train_set = np.asarray([pixels_from_path(dog) for dog in glob.glob('dogs/*')[:SAMPLE_SIZE]])

valid_size = 512

print("loading validation cat images...")
cat_valid_set = np.asarray([pixels_from_path(cat) for cat in glob.glob('cats/*')[-valid_size:]])

print("loading validation dog images...")
dog_valid_set = np.asarray([pixels_from_path(dog) for dog in glob.glob('dogs/*')[-valid_size:]])

x_train = np.concatenate([cat_train_set, dog_train_set])
labels_train = np.asarray([1 for _ in range(SAMPLE_SIZE)]+[0 for _ in range(SAMPLE_SIZE)]) x_valid = np.concatenate([cat_valid_set, dog_valid_set])
labels_valid = np.asarray([1 for _ in range(valid_size)]+[0 for _ in range(valid_size)])

# CNN Implementation

In [ ]:
total_pixels = img_size[0] * img_size[1] * 3
fc_size = 512
inputs = keras.Input(shape=(img_size[1], img_size[0], 3), name = 'ani_image')
x = layers.Flatten(name = 'flattened_img')(inputs) # turn image to vector.
x = layers.Dense(fc_size, activation = 'relu', name = 'first_layer')(x)
outputs = layers.Dense(1, activation = 'sigmoid', name = 'class')(x)

model = keras.Model(inputs = inputs, outputs = outputs)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3e480598-2102-4b3c-a7af-02aec7eef77c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>